In [2]:
import pandas as pd 
import xml.etree.ElementTree as et 
import os
import regex as re
import json
import glob
from tqdm.autonotebook import tqdm

/Users/max/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
wsdlfilelist = []
xsdfilelist = []
# for root, dirs, files in os.walk("../../../Downloads/testcpi"):
for root, dirs, files in os.walk("../../../Downloads/PostNLQAS"):
    wsdlfilelist = wsdlfilelist + glob.glob(root+'/*.wsdl')
    xsdfilelist = xsdfilelist + glob.glob(root+'/*.xsd')

In [4]:
columns = ['filename', 'rootelem', 'rootid', 'rootname', 'path', 'key', 'value', 'name', 'id', 'type', \
           'sourceref', 'targetref', 'processref', 'bpmnelement', 'x', 'y', 'width', 'height', 'bpmntype']

In [5]:
def parse_xsd_XML(xml_file): 
#     print(xml_file)
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    return extractSequence(xroot, xml_file)
    
def extractElementInfos(xml, path, infodict,xml_file):
    for elem in list(xml):
        firsttabs = ""
        rootelem = ""
        if "}" in elem.tag:
            rootelem = elem.tag.split("}", 1)[1]
        else:
            rootelem = elem.tag
        if rootelem == "element":
            minOccurs = elem.attrib.get("minOccurs", None)
            name = elem.attrib.get("name", None)
            maxOccurs = elem.attrib.get("maxOccurs", None)
            elemtype = elem.attrib.get("type", None)
            infodict['path'].append(path)
            infodict['name'].append(name)
            infodict['minOccurs'].append(minOccurs)
            infodict['maxOccurs'].append(maxOccurs)
            infodict['type'].append(elemtype)
            infodict['file'].append(xml_file)
            

def extractSequence(xml, xml_file, infodict = {'file':[],'path': [],'name':[],'minOccurs':[], 'maxOccurs':[],'type':[]},path=""):
    path = path
    infodict = infodict
    if len(list(xml)) > 0:
        for elem in list(xml):
            path = path
            rootelem=""
            if "}" in elem.tag:
                rootelem = elem.tag.split("}", 1)[1]
            else:
                rootelem = elem.tag
            path += "/" + rootelem
            if rootelem == "sequence":
                extractElementInfos(elem, path, infodict,xml_file)
                if len(list(elem))> 0:
                    extractSequence(elem, xml_file, infodict, path)
            else:
                extractSequence(elem, xml_file, infodict, path)
    return infodict

In [1]:
print(parse_xsd_XML(xsdfilelist[0]))

In [9]:
xsddfs = []
pbar = tqdm(total=len(xsdfilelist), leave=False)
pbar.set_description(desc='Processing xsds', refresh=True)
for i in xsdfilelist:
    try:
        dicts = parse_xsd_XML(i)
#     lpbar = tqdm(total=len(dicts), leave=False)
#     lpbar.set_description(desc='Processing dicts', refresh=True)
#     for j in dicts:
        df = pd.DataFrame.from_dict(dicts)
        xsddfs.append(df)
        pbar.update(1)
    except:
        pbar.update(1)
#         lpbar.update(1)
#     lpbar.close()
pbar.close()


In [11]:
df = xsddfs[0]
pbar = tqdm(total=len(xsddfs), leave=False)
pbar.set_description(desc='Processing xsddfs', refresh=True)
pbar.update(1)
for i in range(1, len(xsddfs)):
    df = df.append(xsddfs[i])
    pbar.update(1)
pbar.close()

OSError: [Errno 28] No space left on device

In [13]:
df.to_csv("../generateddata/QASxsd.csv")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25912636 entries, 0 to 75963
Data columns (total 6 columns):
file         object
path         object
name         object
minOccurs    object
maxOccurs    object
type         object
dtypes: object(6)
memory usage: 1.4+ GB


In [12]:
len(wsdlfilelist)

274

In [79]:
def parse_wsdl_XML(wsdl_file): 
#     print(xml_file)
    xtree = et.parse(wsdl_file)
    xroot = xtree.getroot()
    return extractWSDLSequence(xroot, wsdl_file)
    
def extractWSDLElementInfos(xml, path, infodict,xml_file,complextypename):
    for elem in list(xml):
        firsttabs = ""
        rootelem = ""
        if "}" in elem.tag:
            rootelem = elem.tag.split("}", 1)[1]
        else:
            rootelem = elem.tag
        minOccurs = elem.attrib.get("minOccurs", None)
        name = elem.attrib.get("name", None)
        maxOccurs = elem.attrib.get("maxOccurs", None)
        elemtype = elem.attrib.get("type", None)
        infodict['path'].append(path)
        infodict['name'].append(name)
        infodict['min'].append(minOccurs)
        infodict['max'].append(maxOccurs)
        infodict['type'].append(elemtype)
        infodict['filename'].append(xml_file)
        infodict['complextypename'].append(complextypename)
        value = elem.attrib.get("value",None)
        infodict['enum'].append(value)
        enumlength = elem.attrib.get("value",None)
        infodict['enumlength'].append(enumlength)
        maxlength = elem.attrib.get("value",None)
        infodict['maxlength'].append(maxlength)
        minlength = elem.attrib.get("value",None)
        infodict['minlength'].append(minlength)
            
"""
            attribute
            {'name': 'listAgencyID'}
                simpleType
                {}
                    restriction
                    {'base': 'xsd:token'}
                        maxLength
                        {'value': '60'}
                        minLength
                        {'value': '1'}
"""
            

def extractWSDLSequence(xml, xml_file, path="",infodict={'filename':[], 'path':[], 'name':[], 'type':[], 'min':[], 'max':[], 'enum':[], \
                                                                      'enumlength':[], 'maxlength':[], 'minlength':[], 'complextypename':[]},complextypename=None):
    path = path
    infodict = infodict
    if len(list(xml)) > 0:
        for elem in list(xml):
            rootelem=""
            if "}" in elem.tag:
                rootelem = elem.tag.split("}", 1)[1]
            else:
                rootelem = elem.tag
#             print(rootelem)
            path += "/" + rootelem
#             print(pretab+ str(attrib))

            if rootelem == "complexType":
                complextypename = elem.attrib.get('name',None)
                if len(list(elem))> 0:
                        extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            elif rootelem == "group":
                complextypename = elem.attrib.get('name',None)
                if len(list(elem))> 0:
                        extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            elif rootelem == "sequence":
                extractWSDLElementInfos(elem, path, infodict,xml_file,complextypename)
                if len(list(elem))> 0:
                    extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            elif rootelem == "attribute":
                complextypename = elem.attrib.get('name',None)
                if len(list(elem))> 0:
                    extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            elif rootelem == "simpleType":
                extractWSDLElementInfos(elem, path, infodict,xml_file,complextypename)
                if len(list(elem))> 0:
                    extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            elif rootelem == "restriction":
                extractWSDLElementInfos(elem, path, infodict,xml_file,complextypename)
                if len(list(elem))> 0:
                    extractWSDLSequence(elem, xml_file, path, infodict, complextypename)
            else:
                extractWSDLSequence(elem, xml_file, path, infodict, complextypename)

#             extractWSDLSequence(elem,xml_file,path,pretab+"\t",infodict)
    return infodict

In [80]:
columns = ['filename', 'path', 'name', 'type', 'min', 'max', 'enum', 'enumlength', 'maxlength', 'minlength', 'complextypename']

In [81]:
wsdldfs = []
pbar = tqdm(total=len(wsdlfilelist), leave=False)
pbar.set_description(desc='Processing wsdl', refresh=True)
for i in wsdlfilelist:
    dicts = parse_wsdl_XML(i)
#     lpbar = tqdm(total=len(dicts), leave=False)
#     lpbar.set_description(desc='Processing dicts', refresh=True)
#     for j in dicts:
    df = pd.DataFrame.from_dict(dicts)
    pbar.update(1)
    wsdldfs.append(df)
#         lpbar.update(1)
#     lpbar.close()
pbar.close()

In [ ]:
df = wsdldfs[0]
pbar = tqdm(total=len(wsdldfs), leave=False)
pbar.set_description(desc='Processing wsdldfs', refresh=True)
pbar.update(1)
for i in range(1, len(wsdldfs)):
    df = df.append(wsdldfs[i])
    pbar.update(1)
pbar.close()

df.to_csv("../generateddata/wsdl.csv")

In [2]:
# wsdlfilelist[2]